# GR5293 - Proj2 - Group9
## NLP with tweets related to COVID
#### NLP pipeline with sentiment prediction
* Tokenization
    > Split text into tokens(sentences or words), for this question, we split the document into sentence for automatic summarization, and words for sentiment analysis and topic modeling
* Screen out stop words and other meaningless corpus
* Lemmatization
    > Here we only use lemmatization rather than stemming is because lemmatization keeps the interpretability of words with their context. While stemming might lead to incorrect meaning. It is important to make morphological analysis of the words. 
* EDA: wordCloud with different sentiment
    > Identify what poeple with different emotions were considering about
* EDA: Word2vec with Clustering
    > Word2Vec: Effective for detecting the synonymous words or suggesting additional words for a partial sentence

    Clustering methods: K-means + DBScan

    Use all the words in a specific part-of-speech from all the documents (e.g. all nouns / all adj.s)
* (word2vec w/ recommendation)?
* Topic Modeling: Feature extraction by TFIDF + Latent Dirichlet Allocation
    Build a pipeline with KFoldCV to find the best topic number
* Automatic summrization
    > Identify what were most people thinking about or tweeting for
* Sentiment Analysis: Classification for sentiment(5 classes: Neutral / Positive / Extremely Positive / Negative / Extremely Negative)
  
    Potential Model: lightGBM / stacking / BERT?
#### Preprocessing
* One-hot encoding

In [95]:
import numpy
import numpy as np
import pandas as pd
import sklearn
import nltk
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import re
import string
from sklearn import pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%xmode plain
os.getcwd()

Exception reporting mode: Plain


'/Users/kangshuoli/Documents/VScode_workspace/GR5293/EODS-Project2-Group9/EODS_Project2_Group9/doc'

### Read in filtered data

In [96]:
df = pd.read_csv("../data/Corona_NLP_filtered.csv")
df.drop(
    "Unnamed: 0", 
    axis = 1,
    inplace = True
)
df

,OriginalTweet,Sentiment,Tweet_filtered,Word_list,Senten_list,Senten_list_filtered
0,advice Talk to your neighbours family to excha...,Positive,advice talk neighbour family exchange phone nu...,"['advice', 'talk', 'neighbour', 'family', 'exc...",['advice Talk to your neighbours family to exc...,['advice talk to your neighbours family to exc...
1,Coronavirus Australia: Woolworths to give elde...,Positive,coronavirus australia woolworth give elderly d...,"['coronavirus', 'australia', 'woolworth', 'giv...",['Coronavirus Australia: Woolworths to give el...,['coronavirus australia woolworths to give eld...
2,My food stock is not the only one which is emp...,Positive,food stock one empty please dont panic enough ...,"['food', 'stock', 'one', 'empty', 'please', 'd...",['My food stock is not the only one which is e...,['my food stock is not the only one which is e...
3,"Me, ready to go at supermarket during the #COV...",Extremely Negative,ready go supermarket covid outbreak im paranoi...,"['ready', 'go', 'supermarket', 'covid', 'outbr...","['Me, ready to go at supermarket during the #C...",['me ready to go at supermarket during the cov...
4,As news of the regionÂs first confirmed COVID...,Positive,news regionâ  first confirmed covid case came...,"['news', 'regionâ', '\x92', 'first', 'confirme...",['As news of the regionÂ\x92s first confirmed ...,['as news of the regionâ\x92s first confirmed ...
...,...,...,...,...,...,...
44248,Meanwhile In A Supermarket in Israel -- People...,Positive,meanwhile supermarket israel people dance sing...,"['meanwhile', 'supermarket', 'israel', 'people...",['Meanwhile In A Supermarket in Israel -- Peop...,['meanwhile in a supermarket in israel people...
44249,Did you panic buy a lot of non-perishable item...,Negative,panic buy lot nonperishable item echo need foo...,"['panic', 'buy', 'lot', 'nonperishable', 'item...",['Did you panic buy a lot of non-perishable it...,['did you panic buy a lot of nonperishable ite...
44250,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral,asst prof economics talking recent research co...,"['asst', 'prof', 'economics', 'talking', 'rece...","[""Asst Prof of Economics was on talking about ...",['asst prof of economics was on talking about ...
44251,Gov need to do somethings instead of biar je r...,Extremely Negative,gov need somethings instead biar je rakyat ass...,"['gov', 'need', 'somethings', 'instead', 'biar...","[""Gov need to do somethings instead of biar je...",['gov need to do somethings instead of biar je...


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44253 entries, 0 to 44252
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   OriginalTweet         44253 non-null  object
 1   Sentiment             44251 non-null  object
 2   Tweet_filtered        44251 non-null  object
 3   Word_list             44251 non-null  object
 4   Senten_list           44250 non-null  object
 5   Senten_list_filtered  44249 non-null  object
dtypes: object(6)
memory usage: 2.0+ MB


### Data Cleaning
* Most done in data_cleaning.ipynb
* Drop rows with missing values

In [98]:
df.dropna(
    axis = 0, 
    how = "any", 
    inplace = True
)
df.index = np.arange(df.shape[0], dtype = int)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44249 entries, 0 to 44248
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   OriginalTweet         44249 non-null  object
 1   Sentiment             44249 non-null  object
 2   Tweet_filtered        44249 non-null  object
 3   Word_list             44249 non-null  object
 4   Senten_list           44249 non-null  object
 5   Senten_list_filtered  44249 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB


## Word-level Analysis
1. wordCloud
2. word2Vec w/ clustering

## Document-level / Sentence-level Analysis

1. Sentence-level automatic summarization
* Extractive summarization: pick the original sentence which can represent the main focus of the whole document
* Abstractive summarization: generate new sentences for summary
    > Purely extractive summaries often times give better results compared to automatic abstractive summaries. This is because of the fact that abstractive summarization methods cope with problems such as semantic representation, inference and natural language generation which are relatively harder than data-driven approaches such as sentence extraction.

We use frequency-driven approch here. 

##### Extractive method w/ TFIDF
##### Model: Centroid-based summarization

In [139]:
# Centroid-based summarization

# Sentence cleaning
def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def lower(text):
    low_text = text.lower()
    return low_text

def remove_num(text):
    remove = re.sub(r'\d+', '' ,text)
    return remove

def remove_punctuation(text):
    clean_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(clean_list)
    return clean_str

def remove_at(text):
    at = re.compile(r'@.+?\s')
    no_at = re.sub(at, '', text)
    return no_at

stop_words = set(stopwords.words('english'))
wnl = WordNetLemmatizer()

def remove_stop_word(sent):
    word_list = word_tokenize(sent)
    word_list_filtered = [word for word in word_list if word not in stop_words]
    word_list_lemmatized = [wnl.lemmatize(item) for item in word_list_filtered]
    return ' '.join(word_list_lemmatized)



# combine all sentences into one list
sentence_list = []
sentence_list_filtered = []
for i in np.arange(df.shape[0]):
    curr_corpus = df.loc[i, "OriginalTweet"]
    curr_senten_list = sent_tokenize(curr_corpus)
    # lemmatize + stop_words
    curr_senten_list_cleaned = [remove_at(
        remove_urls(
            remove_html(
                remove_num(
                    remove_punctuation(
                        lower(
                            remove_stop_word(item)
                        )
                    )
                )
            )
        )
    ) for item in curr_senten_list]
    sentence_list.extend(curr_senten_list)
    sentence_list_filtered.extend(curr_senten_list_cleaned)


tfidf_sum = TfidfVectorizer(
    norm = "l2", # The cosine similarity between two vectors is their dot product when l2 norm has been applied.
    stop_words = None, # already filtered
    preprocessor = None, 
    lowercase = False, # already lowered
    max_df = 0.01, 
    min_df = 5, 
    ngram_range = (5,10)
)
tfidf_word = tfidf_sum.fit_transform(df["Tweet_filtered"])
vocab_sum = tfidf_sum.get_feature_names()
vocab_sum = [x.replace(' ', '_') for x in vocab_sum]


# sum all the tfidf score for each word
tfidf_word_all_doc = pd.Series(
    np.array(tfidf_word.sum(axis = 0)).reshape(-1, 1).ravel(), 
    index = vocab_sum
)

# set a threshold to filter out the word that are not important
threshold = tfidf_word_all_doc.median()
print(f'TF-IDF threshold: {threshold:0.4f}')

# for each sentence get the sentence centroid score by summing up the word score
def get_centroid_score(sent_):
    score = 0
    word_list = sent_tokenize(sent_)
    for word in word_list:
        if word in vocab_sum: # get rid of some mis-tokenized words
            if tfidf_word_all_doc[word] >= threshold: # TFIDF scores are below a threshold are removed
                score += tfidf_word_all_doc[word]
    return score

# get the sentence score by filtered sentence
sentence_score_dict = {}
for i, curr_senten in enumerate(sentence_list_filtered):
    sentence_score_dict[sentence_list[i]] = get_centroid_score(curr_senten)

TF-IDF threshold: 1.2353


In [141]:
# save the sentence_score_dict
import pickle
with open('centroid_based_sentence_score_ver1.pkl', 'wb') as file:
    pickle.dump(sentence_score_dict, file)
file.close()
'''
ver1:
    norm = "l2", # The cosine similarity between two vectors is their dot product when l2 norm has been applied.
    stop_words = None, # already filtered
    preprocessor = None, 
    lowercase = False, # already lowered
    max_df = 0.01, 
    min_df = 5, 
    ngram_range = (5,10)

''' 


'\nver1:\n    norm = "l2", # The cosine similarity between two vectors is their dot product when l2 norm has been applied.\n    stop_words = None, # already filtered\n    preprocessor = None, \n    lowercase = False, # already lowered\n    max_df = 0.01, \n    min_df = 5, \n    ngram_range = (5,10)\n\n'

In [140]:
# sort by importance value in descending order
sentence_score_dict_sorted = {k: v for k, v in sorted(
    sentence_score_dict.items(), key=lambda item: item[1], 
    reverse = True
)[:50]}

for sentence, score in sentence_score_dict_sorted.items():
    print(sentence, score)

advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order 0
Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P 0
My food stock is not the only one which is empty... 0
PLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. 0
Stay calm, stay safe. 0
#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j 0
Me, ready to go at supermarket during the #COVID19 outbreak. 0
Not because I'm paranoid, but because my food stock is litteraly empty. 0
The #coronavirus is a serious thing, but please, don't panic. 0
It causes shortage...

#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n 0
As news of the

In [104]:
sentence_score_dict

{'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order': 0,
 'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P': 0,
 'My food stock is not the only one which is empty...': 0,
 "PLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need.": 0,
 'Stay calm, stay safe.': 0,
 '#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j': 0,
 'Me, ready to go at supermarket during the #COVID19 outbreak.': 0,
 "Not because I'm paranoid, but because my food stock is litteraly empty.": 0,
 "The #coronavirus is a serious thing, but please, don't panic.": 0,
 'It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtH

2. Document-level sentiment classification